In [1]:
import os

print(os.getcwd())

image = []

with open("donor.hexdump") as f:
    for linelf in f:
        line = linelf.rstrip()
        # if line == "read 4KB"
        #print(line)
        if line == "read 4KB":
            image.append(b"")
            addr = 0
            continue
        if line.startswith("WIP"):
            print(f"{len(image)} {line}")
            continue

        addr_str, rest = line.split(":")
        data_str = rest.strip().split(" ")
        data = [int(s, base=16) for s in data_str]
        if int(addr_str,base=16) != addr:
            raise RuntimeError(f"addr mismatch {addr_str=} {addr=}")
        addr += len(data)
        image[-1] += bytes(data)

c:\Users\dfsmith\Documents\github\arduino\SPI-io
1 WIP=0,WEL=0,BP0=0,BP1=0,S
2 WIP=0,WEL=0,BP0=0,BP1=0,S
3 WIP=0,WEL=0,BP0=0,BP1=0,S


In [2]:
if image[0]==image[2] and image[0]==image[1] and len(image[0])==4096:
    print("match")

match


In [3]:
with open("donor.bin","wb") as f:
     f.write(image[0])